In [1]:
from api import elements
import pandas as pd

In [2]:
elements = elements()

In [5]:
elements.past_seasons.dtypes

season_name                   object
element_code                   int64
start_cost                     int64
end_cost                       int64
total_points                   int64
minutes                        int64
goals_scored                   int64
assists                        int64
clean_sheets                   int64
goals_conceded                 int64
own_goals                      int64
penalties_saved                int64
penalties_missed               int64
yellow_cards                   int64
red_cards                      int64
saves                          int64
bonus                          int64
bps                            int64
influence                     object
creativity                    object
threat                        object
ict_index                     object
starts                         int64
expected_goals                object
expected_assists              object
expected_goal_involvements    object
expected_goals_conceded       object
d